In [1]:
shaking <- function(solution, k) {
  n <- length(solution)
  for (i in 1:k) {
    idx <- sample(1:n, 2)
    solution[idx] <- solution[rev(idx)]
  }
  return(solution)
}


In [2]:
local_search <- function(bins, bin_capacity, binpacking_instance) {
  current_solution <- bins
  best_solution <- current_solution
  best_cost <- binpacking_instance$evaluate(current_solution)
  improved <- TRUE
  
  while (improved) {
    improved <- FALSE
    for (i in 1:(length(current_solution) - 1)) {
      for (j in (i + 1):length(current_solution)) {
        new_solution <- current_solution
        new_solution[c(i, j)] <- new_solution[c(j, i)]
        new_cost <- binpacking_instance$evaluate(new_solution)
        if (new_cost < best_cost) {
          best_solution <- new_solution
          best_cost <- new_cost
          improved <- TRUE
        }
      }
    }
    if (improved) {
      current_solution <- best_solution
    }
  }
  
  return(best_solution)
}

In [3]:
bfd <- function(item_sizes, bin_capacity) {
  sorted_items <- sort(item_sizes, decreasing = TRUE)  # Ordenar los ítems en orden decreciente
  bins <- list()
  
  for (item in sorted_items) {
    placed <- FALSE
    for (bin in bins) {
      if (sum(unlist(bin)) + item <= bin_capacity) {
        bin[[length(bin) + 1]] <- item
        placed <- TRUE
        break
      }
    }
    if (!placed) {
      bins[[length(bins) + 1]] <- list(item)
    }
  }
  
  return(bins)
}


In [4]:
BinPacking <- setRefClass(
  "BinPacking",
  fields = list(
    item_sizes = "numeric",
    bin_capacity = "numeric"
  ),
  methods = list(
    evaluate = function(permutation) {
      bin_capacities <- numeric() 
      current_bin_capacity <- 0  

      for (i in permutation) {
        item_size <- item_sizes[i]
        if (current_bin_capacity + item_size <= bin_capacity) {
          current_bin_capacity <- current_bin_capacity + item_size
        } else {
          bin_capacities <- c(bin_capacities, current_bin_capacity) 
          current_bin_capacity <- item_size  
        }
      }
      bin_capacities <- c(bin_capacities, current_bin_capacity)  
      return(length(bin_capacities)) 
    }
  )
)


In [5]:
read_bin_packing_instance <- function(filename) {
  lines <- readLines(filename)
  P <- as.integer(lines[1])  # Número de instancias de problemas
  problems <- list()
  index <- 2
  for (i in 1:P) {
    problem_id <- trimws(lines[index])
    index <- index + 1
    bin_info <- strsplit(trimws(lines[index]), "\\s+")[[1]]
    bin_capacity <- as.numeric(bin_info[1])
    num_items <- as.integer(bin_info[2])
    best_known_solution <- as.integer(bin_info[3])
    index <- index + 1
    items <- numeric(num_items)
    for (j in 1:num_items) {
      items[j] <- as.numeric(trimws(lines[index]))
      index <- index + 1
    }
    problems[[i]] <- list(
      problem_id = problem_id,
      bin_capacity = bin_capacity,
      num_items = num_items,
      best_known_solution = best_known_solution,
      items = items
    )
  }
  return(problems)
}

In [43]:
# Función Best Fit
best_fit_decreasing <- function(elements, container_capacity) {
  original_indices <- seq_along(elements)
  sorted_indices <- order(elements, decreasing = TRUE)
  elements_sorted <- elements[sorted_indices]
  
  containers <- list()
  assignments <- integer(length(elements)) # Para almacenar las asignaciones de contenedores
  
  for (i in seq_along(elements_sorted)) {
    element <- elements_sorted[i]
    best_fit_index <- -1
    min_space_left <- container_capacity + 1
    
    for (j in seq_along(containers)) {
      space_left <- container_capacity - sum(unlist(containers[[j]]))
      if (space_left >= element && space_left < min_space_left) {
        min_space_left <- space_left
        best_fit_index <- j
      }
    }
    
    if (best_fit_index == -1) {
      containers <- append(containers, list(c(element)))
      best_fit_index <- length(containers)
    } else {
      containers[[best_fit_index]] <- c(containers[[best_fit_index]], element)
    }
    
    assignments[sorted_indices[i]] <- best_fit_index
  }
  
  return(assignments)
}

In [47]:
vns <- function(binpacking_instance, max_iter = 100, max_no_improve = 10) {
  initial_solution <- best_fit_decreasing(binpacking_instance$item_sizes, binpacking_instance$bin_capacity)
  cat("inicial solution", initial_solution)
  best_solution <- initial_solution
  best_cost <- binpacking_instance$evaluate(best_solution)
  no_improve <- 0
  
  for (iter in 1:max_iter) {
    k <- 1
    while (k <= max_no_improve) {
      new_solution <- shaking(best_solution, k)
      new_solution <- local_search(new_solution, binpacking_instance$bin_capacity, binpacking_instance)
      new_cost <- binpacking_instance$evaluate(new_solution)
      
      if (new_cost < best_cost) {
        best_solution <- new_solution
        best_cost <- new_cost
        no_improve <- 0
        k <- 1
      } else {
        k <- k + 1
        no_improve <- no_improve + 1
      }
      
      if (no_improve >= max_no_improve) break
    }
  }
  
  return(best_solution)
}

In [49]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]  # Seleccionar la primera instancia para este ejemplo
instance

Warning message in readLines(filename):
"incomplete final line found on './instances/binpack1.txt'"


$problem_id
[1] "u120_00"

$bin_capacity
[1] 150

$num_items
[1] 120

$best_known_solution
[1] 48

$items
  [1] 42 69 67 57 93 90 38 36 45 42 33 79 27 57 44 84 86 92 46 38 85 33 82 73 49
 [26] 70 59 23 57 72 74 69 33 42 28 46 30 64 29 74 41 49 55 98 80 32 25 38 82 30
 [51] 35 39 57 84 62 50 55 27 30 36 20 78 47 26 45 41 58 98 91 96 73 84 37 93 91
 [76] 43 73 85 81 79 71 80 76 83 41 78 70 23 42 87 43 84 60 55 49 78 73 62 36 44
[101] 94 69 32 96 70 84 58 78 25 80 58 66 83 24 98 60 42 43 43 39

In [50]:
# Función para transformar la solución en una lista de contenedores
transform_solution <- function(solution, item_sizes, bin_capacity) {
  bins <- list()
  current_bin <- numeric()
  current_bin_capacity <- 0
  
  for (i in solution) {
    item_size <- item_sizes[i]
    if (current_bin_capacity + item_size <= bin_capacity) {
      current_bin <- c(current_bin, item_size)
      current_bin_capacity <- current_bin_capacity + item_size
    } else {
      bins <- c(bins, list(current_bin))
      current_bin <- item_size
      current_bin_capacity <- item_size
    }
  }
  bins <- c(bins, list(current_bin))
  return(bins)
}

In [51]:
# Aplicar Best Fit a la instancia leída
elements <- instance$items
container_capacity <- instance$bin_capacity
result <- best_fit_decreasing(elements, container_capacity)
bins <- transform_solution(result, elements, container_capacity)
print(bins)

# Mostrar el resultado
print(result)


[[1]]
[1] 41 70 33

[[2]]
[1] 38 38

[[3]]
[1] 79

[[4]]
[1] 80

[[5]]
[1] 80 42

[[6]]
[1] 49 29 72

[[7]]
[1] 42 36 29

[[8]]
[1] 86 57

[[9]]
[1] 45 73

[[10]]
[1] 80 44 25

[[11]]
[1] 73 30

[[12]]
[1] 69 59

[[13]]
[1] 42 82

[[14]]
[1] 46 69

[[15]]
[1] 46 74 25

[[16]]
[1] 49 38 49

[[17]]
[1] 98

[[18]]
[1] 57 38 30

[[19]]
[1] 55 67

[[20]]
[1] 90 42

[[21]]
[1] 59 25 38

[[22]]
[1] 80 49

[[23]]
[1] 38 32

[[24]]
[1] 98 38

[[25]]
[1] 92 27

[[26]]
[1] 42 85

[[27]]
[1] 28 38 32

[[28]]
[1] 74 69

[[29]]
[1] 93 55

[[30]]
[1] 28 55 45

[[31]]
[1] 69 42

[[32]]
[1] 57 64

[[33]]
[1] 46 32 36 33

[[34]]
[1] 74 64

[[35]]
[1] 84

[[36]]
[1] 70 74

[[37]]
[1] 72 23 46

[[38]]
[1] 33 98

[[39]]
[1] 33 23 82

[[40]]
[1] 49 27 74

[[41]]
[1] 38 79

[[42]]
[1] 82 57

[[43]]
[1] 42 29 44 32

[[44]]
[1] 74

[[45]]
[1] 90 33 25

[[46]]
[1] 93 57

[[47]]
[1] 85 33 28

[[48]]
[1] 82 57

[[49]]
[1] 92

[[50]]
[1] 86

[[51]]
[1] 82 49

[[52]]
[1] 67

[[53]]
[1] 84 55

[[54]]
[1] 41 41

[[55

In [52]:
binpacking_instance <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)
best_solution <- vns(binpacking_instance, 100, 20)
bins <- transform_solution(best_solution, binpacking_instance$item_sizes, binpacking_instance$bin_capacity)
print(bins)

print(best_solution)  
print(paste("Number of bins:", binpacking_instance$evaluate(best_solution)))
beepr::beep()

inicial solution 41 26 22 7 7 12 45 45 34 42 39 30 34 8 39 17 14 9 24 45 15 47 24 37 2 27 10 49 19 32 36 31 47 42 48 25 44 14 48 37 43 3 6 1 27 47 48 45 25 48 46 44 20 18 13 1 21 35 48 46 40 32 5 43 35 43 9 2 10 4 38 19 46 8 11 40 38 16 26 31 30 28 36 22 44 33 28 49 42 13 40 20 12 23 4 34 39 15 46 40 6 33 47 5 29 21 11 35 49 29 18 17 23 42 3 16 43 41 41 44[[1]]
[1] 98 49

[[2]]
[1] 69 38 38

[[3]]
[1] 67 80

[[4]]
[1] 98 42

[[5]]
[1] 49 29 72

[[6]]
[1] 30 42 55

[[7]]
[1] 86 57

[[8]]
[1] 45 73 27

[[9]]
[1] 44 25 73

[[10]]
[1] 30 28 36 42

[[11]]
[1] 33 46 69

[[12]]
[1] 33 74 25

[[13]]
[1] 90 38

[[14]]
[1] 55 90

[[15]]
[1] 57 92

[[16]]
[1] 82 55

[[17]]
[1] 67 49

[[18]]
[1] 42 29 38 38

[[19]]
[1] 80 28 38

[[20]]
[1] 32 98

[[21]]
[1] 38 38 74

[[22]]
[1] 42 85

[[23]]
[1] 70 41 32

[[24]]
[1] 74 69

[[25]]
[1] 93 29 28

[[26]]
[1] 49 92

[[27]]
[1] 33 59 57

[[28]]
[1] 64 46 32

[[29]]
[1] 36 82

[[30]]
[1] 80 64

[[31]]
[1] 79 70

[[32]]
[1] 74 72

[[33]]
[1] 59 46 33

[[3